# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [20]:
import os
import requests
import json
import threading
import gradio as gr

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [21]:
def load_open_ai():
    load_dotenv(override=True)
    api_key = os.getenv('OPENAI_API_KEY')

    if not api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
        print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
            
    openai = OpenAI()
    MODEL_GPT = 'gpt-4o-mini'
 
    return openai, MODEL_GPT


In [22]:
def load_ollama():
    OLLAMA_API = "http://localhost:11434/api/chat"
    HEADERS = {"Content-Type": "application/json"}
    MODEL = "llama3.2"
    
    return OLLAMA_API, HEADERS, MODEL


In [23]:
system_prompt = (
    "You will be given a code snippet. "
    "Your task is to explain what the code does and why it was written that way, using clear and simple language suitable for beginner developers. "
    "Provide a step-by-step explanation and highlight any best practices or potential issues. "
    "You are an experienced and thoughtful tech lead who enjoys mentoring others. "
    "Do not introduce yourself or describe who you are in the response."
)


In [24]:
def build_prompts(code):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "Please explain what this code does and why: " + code}
    ]
    return messages


In [25]:
def explain_code_open_ai(code):
    openai, model = load_open_ai()
    try:
        response = openai.chat.completions.create(
            model=model,
            messages=build_prompts(code),
            stream=False
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        return f"**OpenAI API Error:** {e}"


In [26]:
def explain_code_open_ollama(code):  
    api, headers, model = load_ollama()
    payload = {
        "model": model,
        "messages": build_prompts(code),
        "stream": False
    }

    response = requests.post(api, json=payload, headers=headers)

    if response.status_code == 200:
        data = response.json()  # convert Response to dict
        content = data["message"]["content"]  # extract content
        return content
        # display(Markdown(f"### 🐑 I am **{model.upper()}**\n\n" + content))
    else:
        print(f"Error {response.status_code}: {response.text}")


In [27]:
with gr.Blocks() as view:
    model_choice = gr.Radio(["OpenAI", "Ollama"], label="Select Model", value="OpenAI")
    input_box = gr.Textbox(label="Your message:")
    submit_btn = gr.Button("Submit")
    output_box = gr.Markdown()
    
    def process_with_loading(message, model):
        if model == "OpenAI":
            return explain_code_open_ai(message)
        else:  # Ollama
            return explain_code_open_ollama(message)
    
    submit_btn.click(
        fn=process_with_loading,
        inputs=[input_box, model_choice],
        outputs=[output_box],
    ).then(
        lambda: gr.Button("Submit", interactive=True),
        outputs=[submit_btn]
    )
    
    # This will show the button as "Processing..." while processing
    submit_btn.click(
        lambda: gr.Button("Processing...", interactive=False),
        outputs=[submit_btn]
    )

view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7868

Could not create share link. Missing file: C:\Users\eyala\.cache\huggingface\gradio\frpc\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: C:\Users\eyala\.cache\huggingface\gradio\frpc


In [28]:
# Get gpt-4o-mini to answer, with streaming

In [29]:
# Get Llama 3.2 to answer